<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-26 10:08:42
-------------------
qualified stocks: 94
with latest results: 32
still star stocks: 18


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.99 L
Current:  1.29 C
-------------------
Today PnL: 61.88 K (0.48%)
Current PnL: -16.61 L (-11.78%)
CY Booked + Current PnL: -12.32 L (-8.74%)
-------------------
Total profit:  3.83 L
Total loss:  -20.45 L
-------------------
Total Booked + Current PnL: 14.66 L (12.85%)
Total Booked PnL: 31.27 L (27.42%)
Curr Year Booked PnL: 4.29 L (3.32%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 78.62 L (60.88%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.40 C (108.31%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 8.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBILIFE,1496.49,1924.99,1.57,H-LC,44.68,100632.0,16829.0,7165.0,-0.09,20.08,7.12,28.63,42.0,2.35,0.81,34.47,AR,ATH,INSURANCE
62,RELIANCE,1291.83,1526.00,5.74,X-LC,51.06,156829.0,16020.0,9504.0,0.84,11.38,6.06,18.13,17.0,1.69,1.26,23.83,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,93.62,99954.0,21592.0,12434.0,0.89,27.55,12.44,43.43,71.0,1.74,0.81,33.69,XY24,NTT,MISC
56,PIDILITIND,2504.06,3576.00,-17.01,X-LC,26.60,91941.0,16819.0,15336.0,1.09,22.39,16.68,42.81,11.0,1.10,0.74,15.31,X40,BTT,CHEMICALS
69,SBIN,760.30,863.00,-14.39,M-LC,14.89,210503.0,9024.0,18187.0,0.49,4.48,8.64,13.51,73.0,0.50,1.70,17.64,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VAIBHAVGBL,350.21,521.00,65.29,X-SC,85.11,110414.0,-41577.0,115703.0,5.20,-27.35,104.79,48.77,59.0,-0.36,0.89,26.82,XR,NTT,JEWELLERY
74,SONACOMS,578.05,1006.00,-24.49,M-MC,97.87,97641.0,-3518.0,78406.0,3.84,-3.48,80.30,74.03,151.0,-0.04,0.79,37.22,AR,BTT,AUTO
80,TATAMOTORS,771.32,1065.00,-51.41,M-LC,77.66,216190.0,-9807.0,95859.0,2.73,-4.34,44.34,38.07,68.0,-0.10,1.74,27.27,XY24,NTT,AUTO
5,ALKYLAMINE,2347.98,4567.52,-13.26,X-SC,48.94,80702.0,-20261.0,115702.0,2.27,-20.07,143.37,94.53,60.0,-0.18,0.65,19.65,SR,ATH,CHEMICALS
33,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,87.23,154136.0,3223.0,57231.0,2.21,2.14,37.13,40.06,89.0,0.06,1.24,26.07,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,40.00,M-SC,71.28,145350.0,7772.0,86643.0,-3.48,5.65,59.61,68.62,182.0,0.09,1.17,26.01,X40N,NTT,TEXTILES
20,COFFEEDAY,59.14,80.00,-57.75,L-SC,62.77,63360.0,-50189.0,90237.0,-1.61,-44.20,142.42,35.27,238.0,-0.56,0.51,55.08,XR,NTT,HOTELS
38,INDIAMART,2327.09,4911.36,-52.59,H-SC,47.87,123596.0,260.0,136710.0,-1.52,0.21,110.61,111.05,108.0,0.00,1.00,21.10,AR,ATH,MISC
75,STARHEALTH,551.56,761.00,1.24,M-SC,90.43,165546.0,-34670.0,110701.0,-1.51,-17.32,66.87,37.97,194.0,-0.31,1.33,32.92,XY24,NTT,INSURANCE
16,CAMPUS,294.86,393.00,NaN,NaN,79.79,153914.0,-13861.0,69708.0,-1.17,-8.26,45.29,33.28,175.0,-0.20,1.24,19.16,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1432.81,2275.00,-11.70,X-LC,59.57,219703.0,20542.0,96516.0,1.01,10.31,43.93,58.78,1.0,0.21,1.77,14.71,X40,BTT,IT
81,TCS,3888.13,4998.00,-15.19,X-LC,41.49,212460.0,-20828.0,87427.0,0.75,-8.93,41.15,28.55,2.0,-0.24,1.71,9.58,X40,BTT,IT
50,LTIM,5664.39,7197.33,-5.11,X-LC,89.36,152124.0,-17808.0,63801.0,0.50,-10.48,41.94,27.06,4.0,-0.28,1.23,26.11,X200,ATH,IT
72,SIEMENS,4406.90,7969.85,-9.47,X-LC,86.17,81998.0,-28174.0,117249.0,0.08,-25.57,142.99,80.85,5.0,-0.24,0.66,20.77,AR,ATH,ELECTRICAL
32,HAVELLS,1595.53,2069.16,-4.76,X-LC,10.64,182920.0,-2161.0,57108.0,0.21,-1.17,31.22,29.68,6.0,-0.04,1.47,12.50,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,3.91,H-SC,100.00,120219.0,22572.0,25102.0,1.54,23.12,20.88,48.82,138.0,0.90,0.97,33.55,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,93.62,99954.0,21592.0,12434.0,0.89,27.55,12.44,43.43,71.0,1.74,0.81,33.69,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,1.57,H-LC,44.68,100632.0,16829.0,7165.0,-0.09,20.08,7.12,28.63,42.0,2.35,0.81,34.47,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-17.01,X-LC,26.60,91941.0,16819.0,15336.0,1.09,22.39,16.68,42.81,11.0,1.10,0.74,15.31,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,40.00,M-SC,71.28,145350.0,7772.0,86643.0,-3.48,5.65,59.61,68.62,182.0,0.09,1.17,26.01,X40N,NTT,TEXTILES
57,POLYCAB,5005.95,7541.35,6.01,H-MC,53.72,100606.0,15505.0,27596.0,0.59,18.22,27.43,50.65,92.0,0.56,0.81,26.47,X40N,ATH,CABLES
55,PFIZER,4245.50,6318.30,3.91,H-SC,100.00,120219.0,22572.0,25102.0,1.54,23.12,20.88,48.82,138.0,0.90,0.97,33.55,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,2.28,H-SC,70.21,150824.0,12964.0,24795.0,1.34,9.40,16.44,27.39,118.0,0.52,1.21,30.72,X40,ATH,FMCG
33,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,87.23,154136.0,3223.0,57231.0,2.21,2.14,37.13,40.06,89.0,0.06,1.24,26.07,AR,ATH,AUTO


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-2.00,H-SC,20.21,110093.0,8089.0,36936.0,1.22,7.93,33.55,44.14,111.0,0.22,0.89,25.46,XR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,87.23,154136.0,3223.0,57231.0,2.21,2.14,37.13,40.06,89.0,0.06,1.24,26.07,AR,ATH,AUTO
11,BANKINDIA,113.49,190.00,-31.87,M-MC,13.83,133533.0,4381.0,82684.0,0.24,3.39,61.92,67.42,155.0,0.05,1.08,29.87,XR,NTT,BANKS
93,WIPRO,243.46,420.00,-9.71,M-LC,45.74,154510.0,3565.0,105886.0,0.75,2.36,68.53,72.51,69.0,0.03,1.24,16.33,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.59,H-SC,47.87,123596.0,260.0,136710.0,-1.52,0.21,110.61,111.05,108.0,0.00,1.00,21.10,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.00,-57.75,L-SC,62.77,63360.0,-50189.0,90237.0,-1.61,-44.20,142.42,35.27,238.0,-0.56,0.51,55.08,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-23.22,H-SC,60.64,87495.0,-10485.0,32356.0,0.92,-10.70,36.98,22.32,130.0,-0.32,0.70,26.90,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-330.82,H-SC,80.85,98124.0,-35162.0,119760.0,0.09,-26.38,122.05,63.47,131.0,-0.29,0.79,64.48,XR,NTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,93.62,99954.0,21592.0,12434.0,0.89,27.55,12.44,43.43,71.0,1.74,0.81,33.69,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,1.57,H-LC,44.68,100632.0,16829.0,7165.0,-0.09,20.08,7.12,28.63,42.0,2.35,0.81,34.47,AR,ATH,INSURANCE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,3.91,H-SC,100.00,120219.0,22572.0,25102.0,1.54,23.12,20.88,48.82,138.0,0.90,0.97,33.55,X40,ATH,PHARMA
78,TANLA,917.30,1963.11,-38.60,H-SC,98.94,150310.0,-89105.0,362067.0,1.00,-37.22,240.88,114.01,110.0,-0.25,1.21,37.86,AR,ATH,IT
74,SONACOMS,578.05,1006.00,-24.49,M-MC,97.87,97641.0,-3518.0,78406.0,3.84,-3.48,80.30,74.03,151.0,-0.04,0.79,37.22,AR,BTT,AUTO
18,CERA,8421.60,10420.45,-18.38,X-SC,96.81,84344.0,-25137.0,51121.0,1.92,-22.96,60.61,23.73,64.0,-0.49,0.68,27.66,X40N,ATH,CERAMICS
24,DLF,694.61,1031.70,-15.14,H-LC,95.74,195981.0,21634.0,62969.0,0.55,12.41,32.13,48.53,30.0,0.34,1.58,28.41,AR,ATH,REALTY


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.59
1,25,39.05
2,50,68.66


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    33.11
X40N    14.04
XY25    13.96
X40     13.82
AR      12.19
XR      10.50
X200     1.23
SR       1.17
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.86
M-SC    18.50
X-LC    16.10
H-MC     8.95
M-LC     8.00
H-LC     6.73
X-MC     5.79
M-MC     5.76
X-SC     2.93
L-SC     2.38
L-LC     1.20
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,12.40,-1.04,33.63
BANKS,9.85,-14.82,67.61
IT,9.49,-19.16,85.23
MISC,6.70,-30.31,87.94
INSURANCE,6.19,2.39,26.72
FINANCE,6.18,-24.56,83.71
PAINTS,5.64,-17.59,52.39
AUTO,4.70,-7.17,53.98
ELECTRICAL,4.45,-4.88,58.24


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2866406.0,26
X40N,1209207.0,17
AR,1204027.0,13
XR,1146695.0,14
X40,710145.0,11
XY25,461552.0,10
SR,200056.0,2
X200,63801.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2306902.0,21
M-SC,2021104.0,21
X-LC,695877.0,12
M-MC,552685.0,5
H-MC,515686.0,8
X-SC,344446.0,4
X-MC,316323.0,5
M-LC,313473.0,6
L-SC,301362.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      918652.0      6
M-SC       XY24      748053.0      6
H-SC       AR        558400.0      3
           X40N      501512.0      6
M-SC       X40N      466730.0      6
M-MC       XY24      391595.0      3
X-LC       X40       386816.0      5
M-SC       XR        295318.0      4
H-SC       XR        278441.0      4
M-SC       X40       213877.0      2
H-LC       AR        212123.0      4
M-LC       XY24      189400.0      4
H-MC       XY24      182645.0      2
L-SC       XR        154399.0      2
X-LC       AR        140531.0      2
X-MC       XY24      139710.0      2
M-SC       XY25      119551.0      1
X-SC       XR        115703.0      1
           SR        115702.0      1
H-MC       XR        114264.0      1
X-SC       X40N      113041.0      2
M-LC       XR        105886.0      1
X-LC       XY25      104729.0      4
X-MC       X40N      100328.0      2
M-SC       AR         93221.0      1
           SR         84354.0      1
L-SC       XY24       82848.0      1
M-MC       XR         82684.0      1
           AR         78406.0      1
X-MC       XY25       76285.0      1
H-MC       XY25       74395.0      1
L-SC       AR         64115.0      1
X-LC       X200       63801.0      1
H-MC       X40        59555.0      2
           AR         57231.0      1
H-SC       X40        49897.0      2
L-LC       XY25       40379.0      1
H-LC       XY25       28026.0      1
H-MC       X40N       27596.0      1
M-LC       XY25       18187.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 44.0 seconds
